In [ ]:
import nannyml as nml
from IPython.display import display

# Load synthetic data
reference, analysis, target = nml.load_synthetic_car_loan_dataset()
# display(reference.head())

column_names = [
    'car_value', 'salary_range', 'debt_to_income_ratio', 'loan_length', 'repaid_loan_on_prev_car', 'size_of_downpayment', 'driver_tenure', 'y_pred_proba', 'y_pred'
]

rce = nml.DataReconstructionDriftCalculator(
    column_names=column_names,
    timestamp_column_name='timestamp',
    chunk_size=5000
)
rce.fit(reference)
rcerr = rce.calculate(analysis)

estimator = nml.CBPE(
    y_pred_proba='y_pred_proba',
    y_pred='y_pred',
    y_true='repaid',
    timestamp_column_name='timestamp',
    metrics=['roc_auc',],
    chunk_size=5000,
    problem_type='classification_binary',
    normalize_confusion_matrix='pred',
)
estimator.fit(reference)
estimated = estimator.estimate(analysis)

analysist = analysis.merge(target, left_index=True, right_index=True)
realize = nml.PerformanceCalculator(
    y_pred_proba='y_pred_proba',
    y_pred='y_pred',
    y_true='repaid',
    timestamp_column_name='timestamp',
    problem_type='classification_binary',
    metrics=['roc_auc'],
    chunk_size=5000)
realize.fit(reference)
realized = realize.calculate(analysist)


drift = nml.UnivariateDriftCalculator(
    column_names=column_names,
    treat_as_categorical=['y_pred'],
    timestamp_column_name='timestamp',
    continuous_methods=['jensen_shannon'],
    categorical_methods=['jensen_shannon'],
)
drift.fit(reference)
drifted = drift.calculate(analysis)

In [ ]:
estimated.compare(realized).plot().show()
realized.compare(estimated).plot().show()

In [ ]:
estimated.compare(rcerr).plot().show()
rcerr.compare(estimated).plot().show()

rcerr.compare(realized).plot().show()
realized.compare(rcerr).plot().show()

In [ ]:
drifted.filter(column_names=['salary_range']).compare(rcerr).plot().show()
rcerr.compare(drifted.filter(column_names=['salary_range'])).plot().show()

In [ ]:
drifted.filter(column_names=['salary_range']).compare(estimated).plot().show()
estimated.compare(drifted.filter(column_names=['salary_range'])).plot().show()

drifted.filter(column_names=['salary_range']).compare(realized).plot().show()
realized.compare(drifted.filter(column_names=['salary_range'])).plot().show()